In [21]:
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from ast import literal_eval
from sklearn.model_selection import train_test_split

import os
import random
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [3]:
df_base = pd.read_csv('val_results_no_RAG.csv')
df_rag1 = pd.read_csv('val_results_RAG.csv')
df_rag2 = pd.read_csv('val_results_RAG2.csv')


In [4]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            119 non-null    object
 1   prediction    119 non-null    int64 
 2   is_found      119 non-null    bool  
 3   raw_output    119 non-null    object
 4   paragraph     119 non-null    object
 5   question      119 non-null    object
 6   ground_truth  119 non-null    int64 
dtypes: bool(1), int64(2), object(4)
memory usage: 5.8+ KB


In [11]:
df_base[df_base['prediction'] != df_base['ground_truth']]['id'].tolist()

['generation-for-nlp-1063',
 'generation-for-nlp-501',
 'generation-for-nlp-1048',
 'generation-for-nlp-1137',
 'generation-for-nlp-506',
 'generation-for-nlp-1333',
 'generation-for-nlp-715',
 'generation-for-nlp-564',
 'generation-for-nlp-1056',
 'generation-for-nlp-668',
 'generation-for-nlp-883',
 'generation-for-nlp-472',
 'generation-for-nlp-735',
 'generation-for-nlp-1077',
 'generation-for-nlp-1147',
 'generation-for-nlp-507',
 'generation-for-nlp-484',
 'generation-for-nlp-1374',
 'generation-for-nlp-461',
 'generation-for-nlp-1209',
 'generation-for-nlp-1135',
 'generation-for-nlp-513',
 'generation-for-nlp-743',
 'generation-for-nlp-1303',
 'generation-for-nlp-673',
 'generation-for-nlp-988',
 'generation-for-nlp-491',
 'generation-for-nlp-517',
 'generation-for-nlp-686',
 'generation-for-nlp-1367',
 'generation-for-nlp-921',
 'generation-for-nlp-955',
 'generation-for-nlp-526',
 'generation-for-nlp-436',
 'generation-for-nlp-459']

In [15]:
df_compare = pd.merge(
    df_base[['id', 'prediction', 'ground_truth', 'question']], 
    df_rag1[['id', 'prediction', 'retrieved_chunk']], # RAG에서는 검색된 텍스트도 가져옵니다
    on='id', 
    suffixes=('_base', '_rag')
)

total_count = len(df_compare)
wrong_base_count = len(df_compare[df_compare['prediction_base'] != df_compare['ground_truth']])
acc_base = (df_compare['prediction_base'] == df_compare['ground_truth']).mean()

print(f"📉 [Baseline 분석]")
print(f"- 전체 데이터: {total_count}개")
print(f"- 틀린 개수: {wrong_base_count}개")
print(f"- 정확도: {acc_base:.2%}")
print("-" * 50)


rag_wins = df_compare[
    (df_compare['prediction_base'] != df_compare['ground_truth']) & 
    (df_compare['prediction_rag'] == df_compare['ground_truth'])
]

rag_losses = df_compare[
    (df_compare['prediction_base'] == df_compare['ground_truth']) & 
    (df_compare['prediction_rag'] != df_compare['ground_truth'])
]

both_wrong = df_compare[
    (df_compare['prediction_base'] != df_compare['ground_truth']) & 
    (df_compare['prediction_rag'] != df_compare['ground_truth'])
]

print(f"⚖️ [RAG 효과 분석]")
print(f"✅ RAG로 구제한 문제 (Gain): {len(rag_wins)}개")
print(f"❌ RAG 때문에 망친 문제 (Loss): {len(rag_losses)}개")
print(f"💀 둘 다 못 푼 어려운 문제: {len(both_wrong)}개")
print("=" * 50)

# 5. [범인 색출] RAG 때문에 망친 문제 확인해보기
if len(rag_losses) > 0:
    print(f"🧐 [심층 분석] Baseline은 맞았는데 RAG가 틀린 케이스 (상위 3개)")
    for i, row in rag_losses.head(3).iterrows():
        print(f"\n[ID: {row['id']}] 질문: {row['question']}")
        print(f"🟢 Base 정답: {row['prediction_base']} (정답: {row['ground_truth']})")
        print(f"🔴 RAG 오답: {row['prediction_rag']}")
        print(f"📚 검색된 내용 (앞부분): {str(row['retrieved_chunk'])[:200]}...")
        print("-" * 30)
else:
    print("축하합니다! RAG 때문에 성능이 떨어진 문제는 없습니다.")

📉 [Baseline 분석]
- 전체 데이터: 119개
- 틀린 개수: 35개
- 정확도: 70.59%
--------------------------------------------------
⚖️ [RAG 효과 분석]
✅ RAG로 구제한 문제 (Gain): 6개
❌ RAG 때문에 망친 문제 (Loss): 10개
💀 둘 다 못 푼 어려운 문제: 29개
🧐 [심층 분석] Baseline은 맞았는데 RAG가 틀린 케이스 (상위 3개)

[ID: generation-for-nlp-1281] 질문: 다음 중 은크루마가 아프리카가 "분열에 내재된 모든 위험을 피해야 한다"고 생각하는 이유를 설명하는 것은?
🟢 Base 정답: 4 (정답: 4)
🔴 RAG 오답: 2
📚 검색된 내용 (앞부분): 연합 이사회가 아닌) 아프리카 연합은 연합법에 따라 창설되어, 장차 연방체인 아프리카 경제 공동체로 변화하고 나아가 국제적인 대표 회의를 두는 일개 국가로 발전하고자 한다. 아프리카 연합은 입법부, 사법부, 행정부를 갖춘 의회제 정부로 국가 수반이자 범아프리카 의회 의장인 아프리카 연합의 대통령이 이끈다. 연합 대통령은 범아프리카 의회에서 과반수 득표를 얻...
------------------------------

[ID: generation-for-nlp-822] 질문: 여기에 해당하지 않는 것은 무엇입니까?
🟢 Base 정답: 3 (정답: 3)
🔴 RAG 오답: 4
📚 검색된 내용 (앞부분): 및 지방공무원법 제50조제1항 친절·공정의 의무 국민 전체의 봉사자로서 친절하고 공정하게 직무를 수행하여야 함 국가공무원법 제59조 및 지방공무원법 제51조 종교중립의 의무 종교에 따른 차별 없이 직무를 수행하여야 함 국가공무원법 제59조의2제1항 및 지방공무원법 제51조의2제1항 비밀 엄수의 의무 재직 중과 퇴직 후에 직무상 알게 된 비밀을 엄수하여야 함...
------------------------------

[ID: generation-for-nlp-899] 질문: 이 비

In [20]:

df_merge = pd.merge(
    df_base[['id', 'prediction', 'ground_truth']],
    df_rag1[['id', 'prediction', 'paragraph', 'question', 'retrieved_chunk', 'raw_output']],
    on='id',
    suffixes=('_base', '_rag')
)

# 데이터 타입 통일
for col in ['ground_truth', 'prediction_base', 'prediction_rag']:
    df_merge[col] = df_merge[col].astype(str).str.strip()

# 2. 통계 계산
total = len(df_merge)
rag1_correct_count = (df_merge['prediction_rag'] == df_merge['ground_truth']).sum()
base_correct_count = (df_merge['prediction_base'] == df_merge['ground_truth']).sum()

# Case 분류
# [Loss] Baseline 정답 vs RAG 오답 (가장 중요! 범인 색출)
rag_loss_cases = df_merge[
    (df_merge['prediction_base'] == df_merge['ground_truth']) & 
    (df_merge['prediction_rag'] != df_merge['ground_truth'])
]

# [Gain] Baseline 오답 vs RAG 정답 (RAG가 잘한 것)
rag_win_cases = df_merge[
    (df_merge['prediction_base'] != df_merge['ground_truth']) & 
    (df_merge['prediction_rag'] == df_merge['ground_truth'])
]

print(f"📊 [종합 성적표]")
print(f"1. 전체 데이터 개수 : {total}개")
print(f"2. Base 맞춘 개수   : {base_correct_count}개 (정확도: {base_correct_count/total:.2%})")
print(f"3. RAG1 맞춘 개수   : {rag1_correct_count}개 (정확도: {rag1_correct_count/total:.2%})")
print(f"--------------------------------------------------")
print(f"📉 [RAG 역효과(Loss) 발생]: {len(rag_loss_cases)}개 (Base는 알았는데 RAG 때문에 틀림)")
print(f"📈 [RAG 개선효과(Gain) 발생]: {len(rag_win_cases)}개 (Base는 몰랐는데 RAG 덕분에 맞춤)")
print(f"==================================================\n")

# 3. 상세 출력 함수 (텍스트 안 잘리게)
def print_full_case(row, case_type):
    print(f"🔴 [{case_type} 케이스] ID: {row['id']}")
    print(f"🎯 정답: {row['ground_truth']} | Base 예측: {row['prediction_base']} | RAG 예측: {row['prediction_rag']}")
    print("-" * 30)
    print(f"📄 [지문 (Paragraph)]:\n{row['paragraph']}\n")
    print(f"❓ [질문 (Question)]:\n{row['question']}\n")
    print(f"📚 [검색된 청크 (Retrieved Chunk)]:\n{row['retrieved_chunk']}\n")
    print(f"🤖 [모델 생성 텍스트 (Raw Output)]:\n{row['raw_output']}")
    print("=" * 80 + "\n")

# 4. [Loss 케이스] 집중 분석 출력 (Baseline은 맞췄는데 RAG가 틀린 것)
print("🧐 [집중 분석] RAG가 점수를 깎아먹은 문제들 (Loss Cases)\n")
if len(rag_loss_cases) > 0:
    for _, row in rag_loss_cases.iterrows():
        print_full_case(row, "LOSS (RAG 오답)")
else:
    print("와우! RAG 때문에 틀린 문제가 하나도 없습니다.")

# 5. (선택) Gain 케이스도 보고 싶다면 주석 해제
# print("\n😎 [참고] RAG 덕분에 맞춘 문제들 (Gain Cases)\n")
# for _, row in rag_win_cases.iterrows():
#     print_full_case(row, "WIN (RAG 정답)")

📊 [종합 성적표]
1. 전체 데이터 개수 : 119개
2. Base 맞춘 개수   : 84개 (정확도: 70.59%)
3. RAG1 맞춘 개수   : 80개 (정확도: 67.23%)
--------------------------------------------------
📉 [RAG 역효과(Loss) 발생]: 10개 (Base는 알았는데 RAG 때문에 틀림)
📈 [RAG 개선효과(Gain) 발생]: 6개 (Base는 몰랐는데 RAG 덕분에 맞춤)

🧐 [집중 분석] RAG가 점수를 깎아먹은 문제들 (Loss Cases)

🔴 [LOSS (RAG 오답) 케이스] ID: generation-for-nlp-1281
🎯 정답: 4 | Base 예측: 4 | RAG 예측: 2
------------------------------
📄 [지문 (Paragraph)]:
“아프리카가 세계 평화에 기여할 수 있는 가장 큰 방법은 정치적 연합을 구성해 내부 분열에 내재되어 있는 모든 위험을 피하고 성공적인 연합 구성을 통해 분열된 세계에 모범을 보이는 것입니다. 아프리카 국가들의 연합은 아프리카의 특성을 보다 효과적으로 대표할 것입니다. 이를 통해 크기와 영향력만을 중요시하는 세계에서 존경을 받을 것입니다. 세계 각국은 사하라 사막에서 프랑스가 실시한 원자 폭탄 실험에 대한 아프리카의 반대에 거의 관심을 주지 않았으며, 콩고 공화국이 무정부 상태로 혼란스러운 와중에 콩고에서 유엔이 헌법적 미묘함을 두고 이의를 제기하는 불명예스러운 광경은 열강들이 아프리카의 독립성을 완전히 무시한다는 증거입니다."

❓ [질문 (Question)]:
다음 중 은크루마가 아프리카가 "분열에 내재된 모든 위험을 피해야 한다"고 생각하는 이유를 설명하는 것은?

📚 [검색된 청크 (Retrieved Chunk)]:
연합 이사회가 아닌) 아프리카 연합은 연합법에 따라 창설되어, 장차 연방체인 아프리카 경제 공동체로 변화하고 나아가 국제적인 대표 회의를 두는 일개 국가로 발전하고자 한다. 아프리카 연

In [22]:
ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
dataset = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
dataset.info()

# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

df["choices_len"] = df["choices"].apply(len)
df['choices_len'].value_counts(dropna=False)

df4 = df[df['choices_len'] == 4]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


In [36]:
df[df['id'] == 'generation-for-nlp-822']['paragraph'].tolist()

['모든 공공기관은 다음 윤리지침을 모두 준수합니다.']

In [34]:
df[df['id'] == 'generation-for-nlp-822']['question'].tolist()

['여기에 해당하지 않는 것은 무엇입니까?']

In [35]:
df[df['id'] == 'generation-for-nlp-822']['choices'].tolist()

[['인간에 대한 불필요한 기만 회피',
  '인간과 다른 동물에게 불필요한 고통 방지',
  '컴퓨터를 사용할 수 있는 경우 동물 사용 방지',
  '기밀성 보호']]

In [37]:
df[df['id'] == 'generation-for-nlp-822']['answer'].tolist()

[3]

In [39]:
df_merge = pd.merge(
    df_base[['id', 'prediction', 'ground_truth']],
    df_rag2[['id', 'prediction', 'paragraph', 'question', 'retrieved_chunk', 'raw_output']],
    on='id',
    suffixes=('_base', '_rag')
)

# 데이터 타입 통일
for col in ['ground_truth', 'prediction_base', 'prediction_rag']:
    df_merge[col] = df_merge[col].astype(str).str.strip()

# 2. 통계 계산
total = len(df_merge)
rag1_correct_count = (df_merge['prediction_rag'] == df_merge['ground_truth']).sum()
base_correct_count = (df_merge['prediction_base'] == df_merge['ground_truth']).sum()

# Case 분류
# [Loss] Baseline 정답 vs RAG 오답 (가장 중요! 범인 색출)
rag_loss_cases = df_merge[
    (df_merge['prediction_base'] == df_merge['ground_truth']) & 
    (df_merge['prediction_rag'] != df_merge['ground_truth'])
]

# [Gain] Baseline 오답 vs RAG 정답 (RAG가 잘한 것)
rag_win_cases = df_merge[
    (df_merge['prediction_base'] != df_merge['ground_truth']) & 
    (df_merge['prediction_rag'] == df_merge['ground_truth'])
]

print(f"📊 [종합 성적표]")
print(f"1. 전체 데이터 개수 : {total}개")
print(f"2. Base 맞춘 개수   : {base_correct_count}개 (정확도: {base_correct_count/total:.2%})")
print(f"3. RAG2 맞춘 개수   : {rag1_correct_count}개 (정확도: {rag1_correct_count/total:.2%})")
print(f"--------------------------------------------------")
print(f"📉 [RAG 역효과(Loss) 발생]: {len(rag_loss_cases)}개 (Base는 알았는데 RAG 때문에 틀림)")
print(f"📈 [RAG 개선효과(Gain) 발생]: {len(rag_win_cases)}개 (Base는 몰랐는데 RAG 덕분에 맞춤)")
print(f"==================================================\n")

# 3. 상세 출력 함수 (텍스트 안 잘리게)
def print_full_case(row, case_type):
    print(f"🔴 [{case_type} 케이스] ID: {row['id']}")
    print(f"🎯 정답: {row['ground_truth']} | Base 예측: {row['prediction_base']} | RAG 예측: {row['prediction_rag']}")
    print("-" * 30)
    print(f"📄 [지문 (Paragraph)]:\n{row['paragraph']}\n")
    print(f"❓ [질문 (Question)]:\n{row['question']}\n")
    print(f"📚 [검색된 청크 (Retrieved Chunk)]:\n{row['retrieved_chunk']}\n")
    print(f"🤖 [모델 생성 텍스트 (Raw Output)]:\n{row['raw_output']}")
    print("=" * 80 + "\n")

# 4. [Loss 케이스] 집중 분석 출력 (Baseline은 맞췄는데 RAG가 틀린 것)
print("🧐 [집중 분석] RAG가 점수를 깎아먹은 문제들 (Loss Cases)\n")
if len(rag_loss_cases) > 0:
    for _, row in rag_loss_cases.iterrows():
        print_full_case(row, "LOSS (RAG 오답)")
else:
    print("와우! RAG 때문에 틀린 문제가 하나도 없습니다.")

# 5. (선택) Gain 케이스도 보고 싶다면 주석 해제
# print("\n😎 [참고] RAG 덕분에 맞춘 문제들 (Gain Cases)\n")
# for _, row in rag_win_cases.iterrows():
#     print_full_case(row, "WIN (RAG 정답)")

📊 [종합 성적표]
1. 전체 데이터 개수 : 119개
2. Base 맞춘 개수   : 84개 (정확도: 70.59%)
3. RAG2 맞춘 개수   : 81개 (정확도: 68.07%)
--------------------------------------------------
📉 [RAG 역효과(Loss) 발생]: 8개 (Base는 알았는데 RAG 때문에 틀림)
📈 [RAG 개선효과(Gain) 발생]: 5개 (Base는 몰랐는데 RAG 덕분에 맞춤)

🧐 [집중 분석] RAG가 점수를 깎아먹은 문제들 (Loss Cases)

🔴 [LOSS (RAG 오답) 케이스] ID: generation-for-nlp-1281
🎯 정답: 4 | Base 예측: 4 | RAG 예측: 2
------------------------------
📄 [지문 (Paragraph)]:
“아프리카가 세계 평화에 기여할 수 있는 가장 큰 방법은 정치적 연합을 구성해 내부 분열에 내재되어 있는 모든 위험을 피하고 성공적인 연합 구성을 통해 분열된 세계에 모범을 보이는 것입니다. 아프리카 국가들의 연합은 아프리카의 특성을 보다 효과적으로 대표할 것입니다. 이를 통해 크기와 영향력만을 중요시하는 세계에서 존경을 받을 것입니다. 세계 각국은 사하라 사막에서 프랑스가 실시한 원자 폭탄 실험에 대한 아프리카의 반대에 거의 관심을 주지 않았으며, 콩고 공화국이 무정부 상태로 혼란스러운 와중에 콩고에서 유엔이 헌법적 미묘함을 두고 이의를 제기하는 불명예스러운 광경은 열강들이 아프리카의 독립성을 완전히 무시한다는 증거입니다."

❓ [질문 (Question)]:
다음 중 은크루마가 아프리카가 "분열에 내재된 모든 위험을 피해야 한다"고 생각하는 이유를 설명하는 것은?

📚 [검색된 청크 (Retrieved Chunk)]:
연합 이사회가 아닌) 아프리카 연합은 연합법에 따라 창설되어, 장차 연방체인 아프리카 경제 공동체로 변화하고 나아가 국제적인 대표 회의를 두는 일개 국가로 발전하고자 한다. 아프리카 연합

In [42]:
df_rag2[df_rag2['prediction'] != df_rag2['ground_truth']]

,id,prediction,is_found,raw_output,retrieved_chunk,paragraph,question,ground_truth
1,generation-for-nlp-1063,3,True,<think>\n\n</think>\n\n3,파일:자유권.png|섬네일|4가지 자유사상과 양심의 자유표현과 언론의 자유집회와 결...,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,1
5,generation-for-nlp-501,4,True,<think>\n\n</think>\n\n4,'''사벌주'''(沙伐州) 또는 '''상주'''(尙州)는 신라의 광역 행정구역인 9...,"왕은 사벌주를 상주로 바꾸는 등 9주의 명칭을 개정하고, 군현의 이름도 한 자식으로...",밑줄 친 ‘왕’의 재위 기간에 있었던 일로 옳은 것은?,3
9,generation-for-nlp-1137,4,True,<think>\n\n</think>\n\n4,끼지 않았어야 하였다. 그건 너무나 위험한 것이었다.”고 기록하였다. 미국 육군의 ...,이 지문에 나타난 정보는 무엇입니까? 우리 지도자들은 북으로부터의 침략을 막는 이야...,베트남 전쟁 중 정치적 불화는 다음 중 어떤 전쟁의 정치적 불화와 가장 유사한가요?,2
15,generation-for-nlp-1333,2,True,<think>\n\n</think>\n\n2,_portrait.jpg|니콜로 폴로(베네치아 폴로 삼형제 중 장남)의 아들인 《동...,"그들은 도착하자마자, 주요 관리들이 모두 모인 곳에서 위대한 칸으로부터 명예롭고 은...",다음 중 몽골 제국이 고대 및 중세 세계의 다른 주요 제국들과 매우 달랐다는 점을 ...,1
16,generation-for-nlp-715,3,True,<think>\n\n</think>\n\n3,"'''소비자 물가지수'''(消費者 物價指數,, '''CPI''')는 가정이 소비하기...",노동 통계국이 소비자 물가 지수를 계산하는 데 사용하는 재화 및 용역 비용이 기준 ...,소비자 물가 지수는 어떻게 계산되는가?,4
17,generation-for-nlp-564,4,True,<think>\n\n</think>\n\n4,"의 선함에 대해 깊이 다루고 있으며, 영혼이 정화되어야 함을 역설하고 있으며, 상(...","""1500년 에라스무스가 도버에서 불행을 겪고 집필하여 마운트조이에게 헌정한 작품인...",이 구절에서 에라스무스의 인본주의 유형과 가장 유사한 남부 르네상스 운동은?,1
18,generation-for-nlp-1281,2,True,<think>\n\n</think>\n\n2,"연합 이사회가 아닌) 아프리카 연합은 연합법에 따라 창설되어, 장차 연방체인 아프리...",“아프리카가 세계 평화에 기여할 수 있는 가장 큰 방법은 정치적 연합을 구성해 내부...,"다음 중 은크루마가 아프리카가 ""분열에 내재된 모든 위험을 피해야 한다""고 생각하는...",4
19,generation-for-nlp-1056,1,True,<think>\n\n</think>\n\n1,"'''윌리엄 펜'''(William Penn, 1644년 10월 14일 ~ 1718...",종교적 직업과 예배에서 양심의 자유가 제한된다면 시민의 자유를 최대한 누리더라도 진...,다음 중 펜실베이니아 식민지를 가장 잘 설명한 것은 무엇입니까?,2
21,generation-for-nlp-668,1,True,<think>\n\n</think>\n\n1,에 대한 비교 효과에 대해 탐구했다. 결과는 도시 적응기술의 수행력이 기온의 상승과...,랄프는 같은 제품을 구입하고자 이웃 소도시까지 50마일을 운전하지 않고 근처 정원용...,어떤 유형의 공간적 상호작용을 반영하는가?,3
34,generation-for-nlp-472,4,True,<think>\n\n</think>\n\n4,장경(續藏經)》을 출판했다. 대각국사가 국청사에서 천태교학을 강의한 뒤부터 천태종이...,○불교를 수용하였다. ○태학을 설립하였다.,다음 정책을 시행한 국왕의 재위 기간에 있었던 사실로 옳은 것은?,1


In [47]:
df4[df4['id'] == 'generation-for-nlp-830']['paragraph'].tolist()

['1년 전 비영어권 국가에서 미국으로 이주한 7세 학생은 이주 전의 국가에서는 읽기 수업에서 높은 점수를 받았습니다. 이주 후 일년 동안 그녀는 생활 영어는 잘하게 되었습니다. 영어만 쓰는 2학년이 몇 달 지난 후, 그녀의 선생님은 그녀가 수업 교재인 영어읽기교재(basal reader)로 학습하는 데 큰 어려움을 겪고 있기 때문에 그녀에 대한 평가를 의뢰합니다. 해당 학생에게 실시된 두 가지 영어능력시험에서 해당 학생은 말하기와 듣기에서는 영어만 사용하는 학년 또래의 평균보다 높지만 읽기와 쓰기에서는 평균보다 훨씬 낮은 것으로 나타났습니다. 또한 그녀는 자신의 모국어 읽기 시험에서 학년 또래의 평균보다 훨씬 높은 성적을 보였습니다.']

In [49]:
df4[df4['id'] == 'generation-for-nlp-830']['question'].tolist()

['이러한 정보만으로 다음 중 가장 정확한 해석은 무엇입니까?']

In [50]:
df4[df4['id'] == 'generation-for-nlp-830']['choices'].tolist()

[['가정 환경에서 해당 학생의 모국어를 계속 사용하는 것이 영어 발달을 방해하고 있습니다.',
  '해당 학생의 읽기에서의 어려움은 교과과정에서 더 많은 읽기를 요구하므로 학업문제가 증가할 것이라는 초기 지표입니다.',
  '사회적 능력이 습득되는 여러 상황을 고려하면 해당 학생의 생활영어 능력과 읽기 능력 간의 불일치는 충분히 예상되는 입입니다.',
  '해당 학생의 모국어 읽기 능력과 영어 읽기 능력 간의 차이는 영어가 더 복합적인 것과 관련이 있습니다.']]

In [51]:
df4[df4['id'] == 'generation-for-nlp-693']['paragraph'].tolist()

['사람들이 가까운 시일 내에 특정 제품의 가격이 인상될 것으로 예상하는 경우']

In [52]:
df4[df4['id'] == 'generation-for-nlp-693']['question'].tolist()

['사람들이 가까운 시일 내에 특정 제품의 가격이 인상될 것으로 예상하는 경우에 대한 설명으로 옳은 것은?']

In [53]:
df4[df4['id'] == 'generation-for-nlp-693']['choices'].tolist()

[['이는 현재 제품 수요에는 영향을 미치지 않는다.',
  '이는 제품에 대한 수요를 줄인다.',
  '이는 현재 또는 향후 제품 수요에 영향을 미치지 않는다.',
  '이는 제품에 대한 수요를 늘린다.']]